<a href="https://colab.research.google.com/github/mukhairatil/sentiment-analysis-indoroberta/blob/main/tes_penelitian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-play-scraper pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00


In [2]:
from google_play_scraper import Sort, reviews
import pandas as pd

In [3]:
result, continuation_token = reviews(
    'id.co.bri.brimo',
    lang='id',
    country='id',
    sort=Sort.NEWEST,
    count=500
)

#id.co.bri.brimo → package BRImo
#sort=NEWEST → dari terbaru
#count=500 → ambil dulu banyak, nanti difilter

In [4]:
df = pd.DataFrame(result)
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,b3f0c68b-6a67-45ac-8183-4b9f6ff62686,Farel Ariza,https://play-lh.googleusercontent.com/a/ACg8oc...,sangat mudah digunakan,5,0,2.91.0,2025-12-18 07:49:56,"Hai Sobat BRI, terima kasih atas ulasannya 😊. ...",2025-12-18 07:57:40,2.91.0
1,7344abea-d9c0-4faa-bf58-d7f86451b9c6,gue aje,https://play-lh.googleusercontent.com/a-/ALV-U...,"mBanking terburuk, banyak user termasuk saya m...",1,0,None,2025-12-18 07:44:24,"Hai Sobat BRI, mohon maaf atas kendala yang di...",2025-12-18 08:02:15,None
2,e029306e-6579-4d05-889e-fa567f20eb8c,Ahmad Rizqi ramadhan,https://play-lh.googleusercontent.com/a/ACg8oc...,Tiba-tiba biaya admin jadi 6k yang awalnya 5k,1,0,2.91.0,2025-12-18 07:35:27,"Hai Sobat BRI, mohon maaf atas ketidaknyamanan...",2025-12-18 07:55:05,2.91.0
3,af0101a4-8cbf-44bd-a277-92c62a958d0f,Samuel Mayona,https://play-lh.googleusercontent.com/a-/ALV-U...,berguna buat kita,5,0,2.91.0,2025-12-18 07:34:59,"Hai Sobat BRI, terima kasih atas ulasannya 😊. ...",2025-12-18 07:59:36,2.91.0
4,208708b8-573f-4238-9ec8-6004ef28ca58,Danu Alpriyanto,https://play-lh.googleusercontent.com/a-/ALV-U...,"payah banget gak bisa Screenshot mutasi, padah...",5,0,2.87.1,2025-12-18 07:34:08,"Hai Sobat BRI, mohon maaf atas ketidaknyamanan...",2025-12-18 07:56:35,2.87.1


In [5]:
df['at'] = pd.to_datetime(df['at'])

In [6]:
df_desember = df[
    (df['at'].dt.month == 12) &
    (df['at'].dt.year == 2025)
]

In [7]:
len(df_desember)

500

In [9]:
df_100 = df_desember.head(100)

In [10]:
df_100 = df_100[['content', 'score', 'at']]
df_100.columns = ['review', 'rating', 'date']

In [11]:
df_100.head()

,review,rating,date
0,sangat mudah digunakan,5,2025-12-18 07:49:56
1,"mBanking terburuk, banyak user termasuk saya m...",1,2025-12-18 07:44:24
2,Tiba-tiba biaya admin jadi 6k yang awalnya 5k,1,2025-12-18 07:35:27
3,berguna buat kita,5,2025-12-18 07:34:59
4,"payah banget gak bisa Screenshot mutasi, padah...",5,2025-12-18 07:34:08


In [12]:
len(df_100)

100

In [13]:
df_100.to_csv("brimo_100_desember.csv", index=False)

In [14]:
!ls

brimo_100_desember.csv	sample_data


In [16]:
df.isnull().sum()
#cek missing value

,0
reviewId,0
userName,0
userImage,0
content,0
score,0
thumbsUpCount,0
reviewCreatedVersion,105
at,0
replyContent,6
repliedAt,6


In [20]:
df.columns


Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion'],
      dtype='object')

In [21]:
df = df.drop_duplicates(subset="reviewId")

In [26]:
import re
def clean_text(text):
    # Case folding
    text = text.lower()
    # Hapus mention, link, dan karakter spesial (noise)
    text = re.sub(r'@[A-Za-z0-9_]+|https?://\S+|#\S+|[^\w\s]', '', text)
    # Hapus angka yang tidak penting
    text = re.sub(r'\d+', '', text)
    # Contoh Normalisasi sederhana (bisa kamu tambah sendiri list-nya)
    norm_dict = {'yg': 'yang', 'ga': 'tidak', 'gak': 'tidak', 'bgt': 'banget', 'apps': 'aplikasi'}
    for word, replacement in norm_dict.items():
        text = re.sub(r'\b' + word + r'\b', replacement, text)
    return text.strip()



In [27]:
df['cleaned_text'] = df['content'].apply(clean_text)

In [28]:
df[["content", "cleaned_text"]].head()


,content,cleaned_text
0,sangat mudah digunakan,sangat mudah digunakan
1,"mBanking terburuk, banyak user termasuk saya m...",mbanking terburuk banyak user termasuk saya me...
2,Tiba-tiba biaya admin jadi 6k yang awalnya 5k,tibatiba biaya admin jadi k yang awalnya k
3,berguna buat kita,berguna buat kita
4,"payah banget gak bisa Screenshot mutasi, padah...",payah banget tidak bisa screenshot mutasi pada...


In [29]:
# 3. LABELING
# 1-2: Negatif (0), 3: Netral (1), 4-5: Positif (2)
def create_label(score):
    if score <= 2:
        return 0
    elif score == 3:
        return 1
    else:
        return 2

In [30]:
df['label'] = df['score'].apply(create_label)

In [31]:
df[["score", "label"]].value_counts()


,,count
score,label,
5,2,365
1,0,80
4,2,26
2,0,19
3,1,10


In [33]:
df["label"].value_counts()


,count
label,
2,391
0,99
1,10


In [35]:
df["text_length"] = df["cleaned_text"].apply(len)
df["text_length"].describe()

,text_length
count,500.000000
mean,48.380000
std,69.018452
min,0.000000
25%,7.000000
50%,22.000000
75%,57.000000
max,482.000000


In [36]:
df = df[df["cleaned_text"].str.len() > 0]

In [37]:
df["text_length"].describe()


,text_length
count,497.000000
mean,48.672032
std,69.123918
min,2.000000
25%,7.000000
50%,22.000000
75%,57.000000
max,482.000000
